In [ ]:
import matplotlib.pyplot as plt
import MDAnalysis as mda
import numpy as np
from matplotlib.markers import MarkerStyle as markerstyle
import math
from scipy.interpolate import griddata
import pandas
from scipy import constants
import matplotlib.patches as patches
import matplotlib.cm as cm

In [ ]:
#Energies_File
ener_file = "ADD PATH TO REWEIGHTED ENERGIES FILE HERE"  #Reweighted Energy file For e.g. - "Reweighted_Energies.dat"
dist_file = "ADD PATH TO MASTER DATA FILE"  # Distance file For e.g. - "Final_data.txt"

ener_data = pandas.read_csv(ener_file,comment='#',sep='\t',dtype=np.float64)   #Reweighted Energy file
dist_data = pandas.read_csv(dist_file,comment='#',sep='\t',dtype=np.float64)   #Distance file


#Remove duplicates 
dist_data = dist_data.drop_duplicates(subset=['Timestep'],ignore_index=True)
ener_data = ener_data.drop_duplicates(subset=['Timestep'],ignore_index=True)

#Merge two dataframes
master_data = dist_data.join(ener_data.set_index('Timestep'),on='Timestep',how='inner',lsuffix = '_mda',rsuffix = '_plumed')

## Selecting bound and unbound configurations

In [ ]:


bound_mask = (master_data['d1_mda'] < 12.0) & (master_data['d2_mda'] < 12.0)
# bound_mask = (master_data['RMSD-B'] < 2.0)
unbound_mask = ((master_data['d1_mda'] > 12.0) | (master_data['d2_mda'] > 12.0)) & ((master_data['d1_mda'] < 21.0) & (master_data['d2_mda'] < 21.0))

bd_data = master_data[bound_mask]
unb_data = master_data[unbound_mask]

## Reweighting

Set rwt_bool = True to turn on reweighting

In [ ]:
rwt_bool = False
if rwt_bool:
    bd_rwt = bd_data['rbias']
    unb_rwt = unb_data['rbias']
else:
    bd_rwt = np.ones(len(bd_data))
    unb_rwt = np.ones(len(unb_data))

## Plotting parameters


In [ ]:
f_size=18
leg_size=16
tick_size=16
fig_size= (6,4)



In [ ]:
fig,ax = plt.subplots(figsize=(6,4))

ax.plot(bd_data['d1_mda'],bd_data['d2_mda'],label='Bound',linewidth=1.2,linestyle='',color='steelblue',marker='o',markersize=5)
ax.plot(unb_data['d1_mda'],unb_data['d2_mda'],label='Unbound',linewidth=1.2,linestyle='',color='gold',marker='o',markersize=5)

ax.set_xlabel('d1 (nm)',fontsize=f_size)
ax.set_ylabel('d2 (nm)',fontsize=f_size)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
plt.show()

In [ ]:
ener_col = ['Bond','HP','G96','RA','PDih','ImPDih','Col-Pro-Pro','LJ-Pro-Pro','Col-Pro-W','LJ-Pro-W','Col-W-W','LJ-W-W']
bound_energy = bd_data[ener_col].sum(axis=1)
unbound_energy = unb_data[ener_col].sum(axis=1)

bound_dict={}
unbound_dict={}

fig,ax = plt.subplots(figsize=fig_size)
bonded_terms = ['Bond','HP','G96','RA','PDih','ImPDih']
bonded_sum_bd = bd_data[bonded_terms].sum(axis=1)
bonded_sum_unbd = unb_data[bonded_terms].sum(axis=1)
bound_dict['Bonded'] = [np.mean(bonded_sum_bd), np.std(bonded_sum_bd)]
unbound_dict['Bonded'] = [np.mean(bonded_sum_unbd),np.std(bonded_sum_unbd)]

ax.hist(bonded_sum_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(bonded_sum_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size+2)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('Bonded Interactions',fontsize=f_size)
ax.ticklabel_format(axis='x',style='sci',scilimits=(-3,3))
ax.ticklabel_format(axis='y',style='sci',scilimits=(-2,3))
fig.tight_layout()
plt.savefig('EnergyPlots/3D_Bonded.svg',format='svg',dpi=600,transparent=True)
plt.show()


In [ ]:
def plot_vs_cutoff(plot_data,ener_col,cutoffs,ax,colors=None):
    n_configs=[]
    prev_cutoff = 0
    for cutoff in cutoffs:
        # print(prev_cutoff)
        mask = (plot_data['RMSD-B'] < cutoff) & (plot_data['RMSD-B'] > prev_cutoff)
        data = plot_data[mask]
        energy = data[[ener_col]].sum(axis=1)
        if colors is not None:
            ax.hist(energy,bins=50,alpha=0.8,weights=np.ones(len(energy)),label=f'{cutoff} nm',density=True,color=colors[cutoff])
        else:
            ax.hist(energy,bins=50,alpha=0.8,weights=np.ones(len(energy)),label=f'{cutoff} nm',density=True)

        n_configs.append(len(energy))
        prev_cutoff = cutoff

    return n_configs

        


In [ ]:
fig,ax = plt.subplots(figsize=fig_size)
col_terms = ['Col-Pro-W']
colmb_sum_bd = bd_data[col_terms].sum(axis=1)
colmb_sum_unbd = unb_data[col_terms].sum(axis=1)
bound_dict[col_terms[0]] = [np.mean(colmb_sum_bd), np.std(colmb_sum_bd)]
unbound_dict[col_terms[0]] = [np.mean(colmb_sum_unbd),np.std(colmb_sum_unbd)]

ax.hist(colmb_sum_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(colmb_sum_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('Col (Pro-W) Interactions',fontsize=f_size)
fig.tight_layout()
plt.savefig('EnergyPlots/3D_Col-ProW.svg',format='svg',dpi=600,transparent=True)

fig,ax = plt.subplots(figsize=fig_size)
col_terms = ['Col-Pro-Pro']
colmb_sum_bd = bd_data[col_terms].sum(axis=1)
colmb_sum_unbd = unb_data[col_terms].sum(axis=1)
bound_dict[col_terms[0]] = [np.mean(colmb_sum_bd), np.std(colmb_sum_bd)]
unbound_dict[col_terms[0]] = [np.mean(colmb_sum_unbd),np.std(colmb_sum_unbd)]

ax.hist(colmb_sum_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(colmb_sum_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('Col (Pro-Pro) Interactions',fontsize=f_size)

fig.tight_layout()
plt.savefig('EnergyPlots/3D_Col-ProPro.svg',format='svg',dpi=600,transparent=True)
plt.show()

In [ ]:
col_names = 'LJ-Pro-Pro','Col-Pro-Pro','LJ-Pro-W','Col-Pro-W'

fig,ax=plt.subplots(1,4,figsize=(15,4))

t_size= 18
lw = 3
f_size= 16
for i in range(len(col_names)):
    row_idx = i//2
    col_idx = i%2
    nconfigs = plot_vs_cutoff(master_data,col_names[i],cutoffs=[0.2,0.4,0.6,0.8,1.0,1.4,1.8,2.0],ax=ax[i])
    ax[i].set_title(col_names[i],fontsize=f_size+6)

    print(f'{col_names[i]}: {nconfigs}')


# ax[3].legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2,bbox_to_anchor=(1.5,1))


# ax[0].set_xlim(left=-1450,right=-1100)
# ax[1].set_xlim(left=-13000,right=-10500)
# ax[2].set_xlim(left=-900,right=-200)
# ax[3].set_xlim(left=-17500,right=-13000)


for ax_ in ax.flat:
    # ax_.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
    # ax_.set_ylabel('Frequency',fontsize=f_size)
    
    ax_.tick_params(axis='both',labelsize=t_size)
    ax_.ticklabel_format(axis='x',style='sci',scilimits=(-3,3))
    

fig.tight_layout()

### Energy vs RMSD


In [ ]:
# welltemp = master_data[master_data['Timestep']>100000000]
sorted_df = master_data.sort_values(by='RMSD-B',ascending=True)

hist = np.histogram(sorted_df['RMSD-B'],bins=100)

rmsd_bins = hist[1]

pot_energies = []
pot_energies_rwt = []

for i in range(len(rmsd_bins)-1):
    mask = (sorted_df['RMSD-B'] > rmsd_bins[i]) & (sorted_df['RMSD-B'] < rmsd_bins[i+1])
    pot_energies.append(np.average(sorted_df[mask]['Col-Pro-Pro']))
    pot_energies_rwt.append(np.average(sorted_df[mask]['Col-Pro-Pro'],weights=sorted_df[mask]['rbias']))

fig,ax = plt.subplots(figsize=fig_size)
ax.plot(rmsd_bins[:-1],pot_energies)
ax.plot(rmsd_bins[:-1],pot_energies_rwt)

In [ ]:
print(len(rmsd_bins))

In [ ]:
fig,ax = plt.subplots(figsize=fig_size)
lj_terms = ['LJ-Pro-W']
lj_sum_bd = bd_data[lj_terms].sum(axis=1)
lj_sum_unbd = unb_data[lj_terms].sum(axis=1)
bound_dict[lj_terms[0]] = [np.mean(lj_sum_bd), np.std(lj_sum_bd)]
unbound_dict[lj_terms[0]] = [np.mean(lj_sum_unbd),np.std(lj_sum_unbd)]

ax.hist(lj_sum_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(lj_sum_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('LJ (Pro-W) Interactions',fontsize=f_size)
plt.savefig('EnergyPlots/3D_LJ-ProW.svg',format='svg',dpi=600,transparent=True)
fig.tight_layout()

fig,ax = plt.subplots(figsize=fig_size)
lj_terms = ['LJ-Pro-Pro']
lj_sum_bd = bd_data[lj_terms].sum(axis=1)
lj_sum_unbd = unb_data[lj_terms].sum(axis=1)
bound_dict[lj_terms[0]] = [np.mean(lj_sum_bd), np.std(lj_sum_bd)]
unbound_dict[lj_terms[0]] = [np.mean(lj_sum_unbd),np.std(lj_sum_unbd)]

ax.hist(lj_sum_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(lj_sum_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('LJ (Pro-Pro) Interactions',fontsize=f_size)
fig.tight_layout()
plt.savefig('EnergyPlots/3D_LJ-ProPro.svg',format='svg',dpi=600,transparent=True)
plt.show()

In [ ]:
fig,ax = plt.subplots(figsize=(6,4))
lj_terms='LJ-W-W'
ljW_bd= bd_data[lj_terms]
ljW_unbd = unb_data[lj_terms]
bound_dict[lj_terms] = [np.mean(ljW_bd), np.std(ljW_bd)]
unbound_dict[lj_terms] = [np.mean(ljW_unbd),np.std(ljW_unbd)]

ax.hist(ljW_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(ljW_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
# ax.hist(ljW_bd,bins=50,alpha=0.8,label='Bound',color='steelblue',density=True)
# ax.hist(ljW_unbd,bins=50,alpha=0.8,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.ticklabel_format(axis='x',style='sci',scilimits=(-3,3),useOffset=False)
ax.ticklabel_format(axis='y',style='sci',scilimits=(-2,3))
# ax.xaxis.set_major_formatter(FormatStrFormatter('%.2f'))
ax.set_title('LJ Energy (W-W) Interactions',fontsize=f_size)
fig.tight_layout()
plt.savefig('EnergyPlots/3D_LJ-WW.svg',format='svg',dpi=600,transparent=True)
plt.show()

fig,ax = plt.subplots(figsize=(6,4))
col_terms = 'Col-W-W'
colW_bd= bd_data[col_terms]
colW_unbd = unb_data[col_terms]
bound_dict[col_terms] = [np.mean(colW_bd), np.std(colW_bd)]
unbound_dict[col_terms] = [np.mean(colW_unbd),np.std(colW_unbd)]

ax.hist(colW_bd,bins=50,alpha=0.8,weights=bd_rwt,label='Bound',color='steelblue',density=True)
ax.hist(colW_unbd,bins=50,alpha=0.8,weights=unb_rwt,label='Unbound',color='gold',density=True)
ax.legend(fontsize=leg_size,fancybox=True,framealpha=0.6,markerscale=1.2)
ax.tick_params(axis='both',labelsize=tick_size)
ax.ticklabel_format(axis='y',style='sci',scilimits=(-2,3))
ax.set_xlabel('Energy (kJ/mol)',fontsize=f_size)
ax.set_ylabel('Frequency',fontsize=f_size)
ax.set_title('Columb Energy (W-W) Distribution',fontsize=f_size)
fig.tight_layout()
plt.savefig('EnergyPlots/3D_Col-WW.svg',format='svg',dpi=600,transparent=True)
plt.show()

In [ ]:
"""
3D
"""
"""
------- Defining Constants ------------
"""
k = constants.value('Boltzmann constant')
Ava_no = constants.value('Avogadro constant')
temp = 310
kbt = (k*temp*Ava_no)

#Add new column to data frame called 'State'

col_names = ['LJ-Pro-Pro','LJ-Pro-W','Col-Pro-Pro','Col-Pro-W', 'LJ-W-W','Col-W-W']

rwt_bool = True

bound_mask = (master_data['d1_mda'] < 12.0) & (master_data['d2_mda'] < 12.0)
cry_mask = (master_data['RMSD-B'] < 2.0)
unbound_mask = ((master_data['d1_mda'] > 12.0) | (master_data['d2_mda'] > 12.0)) & ((master_data['d1_mda'] < 19.0) & (master_data['d2_mda'] < 19.0))

bd_energies = {}
unb_energies = {}
cry_energies = {}

std_bd = {}
std_unb = {}
std_cry = {}
for col in col_names:
    if rwt_bool:
        bd_energies[col] = np.average(master_data[col][bound_mask],weights=np.exp(master_data['rbias'][bound_mask]/kbt))
        unb_energies[col] = np.average(master_data[col][unbound_mask],weights=np.exp(master_data['rbias'][unbound_mask]/kbt))
        cry_energies[col] = np.average(master_data[col][cry_mask],weights=np.exp(master_data['rbias'][cry_mask]/kbt))
        
        
        std_bd[col] = np.sqrt(np.average((master_data[col][bound_mask]-bd_energies[col])**2,weights=np.exp(master_data['rbias'][bound_mask]/kbt)))
        std_unb[col] = np.sqrt(np.average((master_data[col][unbound_mask]-unb_energies[col])**2,weights=np.exp(master_data['rbias'][unbound_mask]/kbt)))
        std_cry[col] = np.sqrt(np.average((master_data[col][cry_mask]-cry_energies[col])**2,weights=np.exp(master_data['rbias'][cry_mask]/kbt)))

        
    else:
        bd_energies[col] = np.average(master_data[col][bound_mask])
        unb_energies[col] = np.average(master_data[col][unbound_mask])
        cry_energies[col] = np.average(master_data[col][cry_mask])

        std_bd[col] = np.std(master_data[col][bound_mask])
        std_unb[col] = np.std(master_data[col][unbound_mask])
        std_cry[col] = np.std(master_data[col][cry_mask])
        
bonded_terms = ['Bond','HP','G96','RA','PDih','ImPDih']
bonded_sum_bd = np.average(master_data[bonded_terms][bound_mask].sum(axis=1),weights=np.exp(master_data['rbias'][bound_mask]/kbt))
bonded_sum_unbd = np.average(master_data[bonded_terms][unbound_mask].sum(axis=1),weights=np.exp(master_data['rbias'][unbound_mask]/kbt))
bonded_sum_cry = np.average(master_data[bonded_terms][cry_mask].sum(axis=1),weights=np.exp(master_data['rbias'][cry_mask]/kbt))

std_bonded_bd = np.sqrt(np.average((master_data[bonded_terms][bound_mask].sum(axis=1)-bonded_sum_bd)**2,weights=np.exp(master_data['rbias'][bound_mask]/kbt)))
std_bonded_unb = np.sqrt(np.average((master_data[bonded_terms][unbound_mask].sum(axis=1)-bonded_sum_unbd)**2,weights=np.exp(master_data['rbias'][unbound_mask]/kbt)))
std_bonded_cry = np.sqrt(np.average((master_data[bonded_terms][cry_mask].sum(axis=1)-bonded_sum_cry)**2,weights=np.exp(master_data['rbias'][cry_mask]/kbt)))
                        

col_names.append('Bonded')
bd_energies['Bonded'] = bonded_sum_bd
unb_energies['Bonded'] = bonded_sum_unbd
cry_energies['Bonded'] = bonded_sum_cry

std_bd['Bonded']=std_bonded_bd
std_unb['Bonded']=std_bonded_unb
std_cry['Bonded']=std_bonded_cry



In [ ]:

print("--------------------- 3D --------------------")
print("\t\t Average \t\t")
print("---------------------------------------------")
print("%-10s\t" %('State'),end='')
for col in col_names:
    print("%-10s\t" % (col), end='')
print("%-10s\t" %('Total'))
print("---------------------------------------------")

print("%-10s\t" %('Bound'),end='')
dU_sum = 0
for col in col_names:
    dU = bd_energies[col]-unb_energies[col]
    print("%-8.2f\t" % (bd_energies[col]), end='')
    dU_sum += dU
print("%-8.2f" % dU_sum)
print()

print("%-10s\t" %('Specific'),end='')
dU_sum = 0
for col in col_names:
    dU = cry_energies[col]-unb_energies[col]
    print("%-8.2f\t" % (cry_energies[col]), end='')
    dU_sum += dU
print("%-8.2f" % dU_sum)
print()

print("%-10s\t" %('Unbound'),end='')
dU_sum = 0
for col in col_names:
    dU = unb_energies[col]-unb_energies[col]
    print("%-8.2f\t" % (unb_energies[col]), end='')
    dU_sum += dU
print("%-8.2f" % dU_sum)
print()

print("---------------------------------------------")
print("\t\t Standard Deviation \t")
print("---------------------------------------------")
print("%-10s\t" %('State'),end='')
for col in col_names:
    print("%-10s\t" % (col), end='')

print()
print("---------------------------------------------")
print("%-10s\t" %('Bound'),end='')
for col in col_names:
    print("%-8.2f\t" % (std_bd[col]), end='')

print()
print("%-10s\t" %('Specific'),end='')
for col in col_names:
    print("%-8.2f\t" % (std_cry[col]), end='')

print()
print("%-10s\t" %('Unbound'),end='')
for col in col_names:
    print("%-8.2f\t" % (std_unb[col]), end='')



In [ ]:
print(np.sum(bound_mask))
print(np.sum(cry_mask))
print(np.sum(unbound_mask))

def get_true_blocks(arr):
    blocks = []
    start = None
    for i, val in enumerate(arr):
        if val and start is None:
            start = i
        elif not val and start is not None:
            blocks.append((start, i))
            start = None
    if start is not None:  # Last block if it ends in True
        blocks.append((start, len(arr)))
    return blocks

print(len(master_data['Timestep']))

In [ ]:


def calculate_block_energies(data,block_ind):
    col_names = ['LJ-Pro-Pro','LJ-Pro-W','Col-Pro-Pro','Col-Pro-W', 'LJ-W-W','Col-W-W']
    bd_energies = {}
    unb_energies = {}
    cry_energies = {}

    bound_mask = np.array([False]*len(data))
    cry_mask = np.array([False]*len(data))
    unbound_mask = np.array([False]*len(data))

    cry_mask[block_ind[0]] = True
    bound_mask[block_ind[1]] = True
    unbound_mask[block_ind[2]] = True

    for col in col_names:
        if rwt_bool:
            bd_energies[col] = np.average(data[col][bound_mask],weights=np.exp(data['rbias'][bound_mask]/kbt))
            unb_energies[col] = np.average(data[col][unbound_mask],weights=np.exp(data['rbias'][unbound_mask]/kbt))
            cry_energies[col] = np.average(data[col][cry_mask],weights=np.exp(data['rbias'][cry_mask]/kbt))
            
        else:
            bd_energies[col] = np.average(data[col][bound_mask])
            unb_energies[col] = np.average(data[col][unbound_mask])
            cry_energies[col] = np.average(data[col][cry_mask])
  
    bonded_terms = ['Bond','HP','G96','RA','PDih','ImPDih']
    bonded_sum_bd = np.average(data[bonded_terms][bound_mask].sum(axis=1),weights=np.exp(data['rbias'][bound_mask]/kbt))
    bonded_sum_unbd = np.average(data[bonded_terms][unbound_mask].sum(axis=1),weights=np.exp(data['rbias'][unbound_mask]/kbt))
    bonded_sum_cry = np.average(data[bonded_terms][cry_mask].sum(axis=1),weights=np.exp(data['rbias'][cry_mask]/kbt))


    col_names.append('Bonded')
    bd_energies['Bonded'] = bonded_sum_bd
    unb_energies['Bonded'] = bonded_sum_unbd
    cry_energies['Bonded'] = bonded_sum_cry

    return cry_energies,bd_energies,unb_energies
   

In [ ]:
# Get blocks of True values
import random

random.seed(4354353)

bound_indices = np.where(bound_mask)[0]
unbound_indices = np.where(unbound_mask)[0]
cry_indices = np.where(cry_mask)[0]

n_blocks = 5

# random.shuffle(bound_indices)
# random.shuffle(unbound_indices)
# random.shuffle(cry_indices)

bound_blocks = np.array_split(bound_indices,n_blocks)
unbound_blocks = np.array_split(unbound_indices,n_blocks)
cry_blocks = np.array_split(cry_indices,n_blocks)

block_energy_cry = []
block_energy_bd = []
block_energy_unb = []
for i in range(n_blocks):

    cry_energy,bd_energy,unb_energy = calculate_block_energies(master_data,[cry_blocks[i],bound_blocks[i],unbound_blocks[i]])

    block_energy_cry.append(cry_energy)
    block_energy_bd.append(bd_energy)
    block_energy_unb.append(unb_energy)

for col in col_names:

    bd_col = [block[col] for block in block_energy_bd]
    unb_col = [block[col] for block in block_energy_unb]
    cry_col = [block[col] for block in block_energy_cry]
    # print(col)
    # print(bd_col)

    bd_avg = np.mean(bd_col)
    bd_sem = np.std(bd_col)/np.sqrt(n_blocks)

    unb_avg = np.mean(unb_col)
    unb_sem = np.std(unb_col)/np.sqrt(n_blocks)

    cry_avg = np.mean(cry_col)
    cry_sem = np.std(cry_col)/np.sqrt(n_blocks)

    print(f"{col} : Bound: {bd_avg:.2f} +/- {bd_sem:.2f} Unbound: {unb_avg:.2f} +/- {unb_sem:.2f} Cry: {cry_avg:.2f} +/- {cry_sem:.2f}")   


